In [10]:
import ee

# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

import geemap
import json
from ipyleaflet import GeoJSON, Marker, MarkerCluster
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from skimage import exposure
import cv2
import os
from dl_landsat import LANDSAT_8, apply_scale_factors_8, mask_clouds, export_to_drive

In [15]:
CITY = ee.Geometry.Point([129.3145, 36.0030]) # Coord format: E, N 

YEAR = 2013

Map = geemap.Map()

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [16]:
ls_read = (ee.ImageCollection(LANDSAT_8)
           .map(apply_scale_factors_8)
           .filterBounds(CITY)
           .filterDate(f'{YEAR}-01-01', f'{YEAR}-12-31')
           .map(mask_clouds)
           )
ls_median = ls_read.median()

data_available = ls_median.getInfo() is not None

print("Data Available: ", data_available)
if data_available:
    print("Number of exported Months:")
    print(ls_read.size().getInfo())
    time_stamps = ls_read.aggregate_array("system:time_start").getInfo()
    dates = list(
        map(lambda x: datetime.datetime.fromtimestamp(x / 1e3).strftime("%Y/%m/%d %H:%M"), time_stamps))
    print("Images Exported:")
    for date in dates:
        print(date)
    vis_params_wc = {"bands": ['Map']}
    vis_params_rgb = {'bands': ['B3', "B2", "B1"], "min": 0, "max": 0.3, 'opacity': 1.0}
    Map.centerObject(CITY, 14)
    Map.addLayer(ls_median, vis_params_rgb, f"Satelite Image")
else:
    print("Error Couldn't find datapoint")

Data Available:  True
Number of exported Months:
14
Images Exported:
2013/04/11 11:03
2013/05/13 11:01
2013/06/30 11:01
2013/07/16 11:01
2013/08/01 11:01
2013/08/17 11:01
2013/09/02 11:01
2013/09/18 11:01
2013/10/04 11:00
2013/10/20 11:00
2013/11/05 11:00
2013/11/21 11:00
2013/12/07 11:00
2013/12/23 11:00
